In [1]:
# Imports
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image

In [92]:
# Convolutional neural net class
# This needs work. I think the brunt of the work here is getting from our, currently 3x120x120, jgps and 4x120x120 pngs to
# a 151 length result vector through convolution. I think the LeNet5 diagram has a good indication of what
# to do but if any of you have more of an idea let me know and put something in the README.md file.
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.sm = nn.Softmax()
        self.pool1 = nn.AvgPool2d(3, stride=2)
        self.pool2 = nn.AvgPool2d(3, stride=2)
        self.pool3 = nn.MaxPool2d(3, stride=2)
        self.conv1 = nn.LazyConv2d(96, 11, stride=4)
        self.conv2 = nn.LazyConv2d(256, 7, padding=2)
        self.conv3 = nn.LazyConv2d(384, 5, padding=1)
        self.conv4 = nn.LazyConv2d(384, 5, padding=1)
        self.flat = nn.Flatten(0)
        self.lin1 = nn.Linear(3456, 151, )
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(F.relu(x))
        x = self.conv2(x)
        x = self.pool2(F.relu(x))
        x = self.conv3(x)
        x = self.conv4(F.relu(x))
        x = self.pool3(F.relu(x))
        x = self.flat(x)
        x = self.lin1(x)
        return F.softmax(F.relu(x))

In [41]:
# Normalizing so that we can process images easily
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        transforms.Resize([224, 224]),
    ]
)

def one_hot_tensor(x):
    l = [0.0 for i in range(151)]
    l[x - 1] = 1.0
    return torch.tensor(l)

In [79]:
# Initializations
cnn = CNN()
# cnn = torchvision.models.AlexNet(num_classes=151)
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01, momentum=0.9)
# Paths to images for training data in our repo AKA pokedex numbers
pdns = sorted([int(x) for x in os.listdir("train/")])

In [44]:
def train():
    images = {
        label: [
            "train/" + str(label) + "/" + image_name
            for image_name in os.listdir("train/" + str(label) + "/")
        ]
        for label in pdns
    }

    count = 0
    rl = 0.0
    for i in pdns:
        print("Training PDN: " + str(i))
        inputs, labels = (
            [transform(Image.open(path).convert("RGB")) for path in images[i]],
            [one_hot_tensor(i) for x in images[i]],
        )

        optimizer.zero_grad()
        for j in range(len(inputs)):
            x = inputs[j]
            y = labels[j]
            out = cnn(x)
            # print(cnn.conv1.parameters())
            print(out, torch.argmax(out))
            opt = loss(out, y)
            opt.backward()
            optimizer.step()
            count += 1
            rl += opt.item()
            if count % 50 == 0:
                print(str(count) + " with loss of " + str(rl))
                rl = 0.0

    torch.save(cnn.state_dict(), "cnn.pth")

In [87]:
def neo_train():
    test_images = {
        label: [
            "train/" + str(label) + "/" + image_name
            for image_name in os.listdir("train/" + str(label) + "/")
        ]
        for label in pdns
    }

    testx = []
    for i in pdns:
        for j in range(len(test_images[i])):
            testx.append([test_images[i][j], torch.LongTensor([i - 1.0])])

    print(len(testx))
    random.shuffle(testx)
    print(len(testx))

    count = 0
    rl = 0.0
    optimizer.zero_grad()
    for XY in testx:
        # print(XY[0])
        res = cnn(transform(Image.open(XY[0]).convert("RGB")))
        l = loss(res, XY[1][0])
        print(torch.argmax(res).item(), XY[1][0].item(), l.item())
        l.backward()
        optimizer.step()
        count += 1
        rl += l.item()
        if count % 100 == 0:
            print(str(count) + " with loss of " + str(rl))
            rl = 0.0
            optimizer.zero_grad()

    torch.save(cnn.state_dict(), "cnn.pth")

In [88]:
def test():
    cnn.load_state_dict(torch.load("cnn.pth"))
    test_images = {
        label: [
            "test/" + str(label) + "/" + image_name
            for image_name in os.listdir("test/" + str(label) + "/")
        ]
        for label in pdns
    }

    count = 0
    correct = 0
    for i in pdns:
        inputs, labels = (
            [transform(Image.open(path).convert("RGB")) for path in test_images[i]],
            [one_hot_tensor(i) for x in test_images[i]],
        )
        
        print("Testing PDN: " + str(i))

        prev_count = count
        prev_correct = correct
        for j in range(len(inputs)):
            x = inputs[j]
            y = labels[j]
            res = cnn(x)
            # print(res)
            print("True: " + str(torch.argmax(y).item()) + " Guess: " + str(torch.argmax(res).item()))
            if torch.argmax(res) == torch.argmax(y):
                correct += 1
            count += 1

        print(
            "For PDN: "
            + str(i)
            + " got "
            + str(correct - prev_correct)
            + " out of "
            + str(count - prev_count)
            + " correct",
        )

In [94]:
cnn = "A"
cnn = CNN()
neo_train()

/tmp/ipykernel_2804/1933967476.py:13: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  testx.append([test_images[i][j], torch.LongTensor([i - 1.0])])


4923
4923


/tmp/ipykernel_2804/2592652201.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(F.relu(x))


69 92 5.017336368560791
149 93 5.017368316650391
149 149 5.016597747802734
149 52 5.017221927642822
149 123 5.017141819000244
130 23 5.017319202423096
149 6 5.0171427726745605
149 61 5.017178535461426
119 134 5.017364025115967
86 63 5.017348766326904
149 34 5.017387390136719
130 135 5.01733922958374
130 52 5.017303943634033
130 140 5.017377853393555
119 16 5.017364025115967
102 42 5.017284870147705
149 150 5.0172882080078125
149 18 5.017344951629639
149 64 5.017247676849365
149 101 5.0172858238220215
149 117 5.017370700836182
130 127 5.017364978790283
149 108 5.017337322235107
130 3 5.017363548278809
130 79 5.0173563957214355
130 116 5.016959190368652
119 13 5.017199516296387
149 34 5.01737642288208
130 43 5.0173659324646
69 61 5.017155170440674
149 101 5.01732063293457
130 3 5.017378330230713
130 23 5.017371654510498
130 26 5.017277717590332
130 111 5.017141342163086
119 76 5.017033100128174
149 8 5.0173444747924805
149 116 5.017038345336914
149 55 5.017265319824219
130 84 5.017365932

130 67 5.017354965209961
130 64 5.017355918884277
130 146 5.017385005950928
130 44 5.0173563957214355
149 99 5.017118453979492
149 94 5.017268657684326
130 91 5.017299652099609
149 138 5.017344951629639
119 63 5.017353534698486
149 150 5.017238616943359
149 142 5.017364978790283
149 108 5.0173258781433105
149 133 5.017290115356445
149 54 5.01737117767334
149 30 5.017329216003418
130 148 5.017288684844971
119 21 5.0171799659729
149 97 5.017327785491943
130 87 5.017301082611084
149 38 5.017280101776123
149 100 5.017343997955322
130 149 5.0168681144714355
149 97 5.0173540115356445
86 139 5.0172624588012695
149 98 5.017330646514893
86 46 5.017366886138916
149 56 5.017344951629639
130 44 5.017354488372803
40 24 5.017378330230713
130 43 5.017337322235107
130 97 5.017347812652588
149 60 5.0172438621521
149 59 5.017389297485352
149 92 5.0173659324646
119 96 5.017264366149902
149 83 5.017334938049316
149 138 5.017347812652588
149 27 5.017287731170654
149 39 5.017369747161865
40 24 5.01736831665

130 74 5.017373561859131
130 135 5.017232418060303
40 27 5.017275810241699
149 37 5.0173540115356445
149 39 5.01734733581543
149 93 5.017337799072266
149 112 5.017373561859131
130 37 5.0173773765563965
149 1 5.0173516273498535
149 77 5.017151355743408
149 85 5.017362117767334
149 2 5.017329216003418
149 56 5.01735258102417
130

KeyboardInterrupt: 

In [93]:
cnn(transform(Image.open("test/1/0.jpg").convert("RGB")))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x3456 and 2304x151)

In [63]:
test()

Testing PDN: 1
True: 0 Guess: 0
True: 0 Guess: 0
True: 0 Guess: 0
True: 0 Guess: 0
True: 0 Guess: 0
For PDN: 1 got 5 out of 5 correct
Testing PDN: 2
True: 1 Guess: 0
True: 1 Guess: 0
True: 1 Guess: 0
True: 1 Guess: 0
True: 1 Guess: 0
For PDN: 2 got 0 out of 5 correct


/tmp/ipykernel_2804/2293414919.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(F.relu(x))
/tmp/ipykernel_2804/204415514.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  res = F.softmax(cnn(x))


Testing PDN: 3
True: 2 Guess: 0
True: 2 Guess: 0
True: 2 Guess: 0
True: 2 Guess: 0
True: 2 Guess: 0
True: 2 Guess: 0
True: 2 Guess: 0
For PDN: 3 got 0 out of 7 correct
Testing PDN: 4
True: 3 Guess: 0
True: 3 Guess: 0
True: 3 Guess: 0
True: 3 Guess: 0
True: 3 Guess: 0
True: 3 Guess: 0
For PDN: 4 got 0 out of 6 correct
Testing PDN: 5
True: 4 Guess: 0
True: 4 Guess: 0
True: 4 Guess: 0
True: 4 Guess: 0
True: 4 Guess: 0
For PDN: 5 got 0 out of 5 correct
Testing PDN: 6
True: 5 Guess: 0
True: 5 Guess: 0
True: 5 Guess: 0
True: 5 Guess: 0
For PDN: 6 got 0 out of 4 correct
Testing PDN: 7
True: 6 Guess: 0
True: 6 Guess: 0
True: 6 Guess: 0
True: 6 Guess: 0
True: 6 Guess: 0
True: 6 Guess: 0
For PDN: 7 got 0 out of 6 correct
Testing PDN: 8
True: 7 Guess: 0
True: 7 Guess: 0
True: 7 Guess: 0
True: 7 Guess: 0
True: 7 Guess: 0
For PDN: 8 got 0 out of 5 correct
Testing PDN: 9
True: 8 Guess: 0
True: 8 Guess: 0
True: 8 Guess: 0
True: 8 Guess: 0
True: 8 Guess: 0
For PDN: 9 got 0 out of 5 correct
Testing PDN

KeyboardInterrupt: 